Copy Contents to starter.py for submission

In [1]:
# Imports
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import math
import time
import numpy as np
import sys
import argparse
import os

In [2]:
# Encoding and decoding
def decode(vocab,corpus):
    
    text = ''
    for i in range(len(corpus)):
        wID = corpus[i]
        text = text + vocab[wID] + ' '
    return(text)

def encode(words,text):
    corpus = []
    tokens = text.split(' ')
    for t in tokens:
        try:
            wID = words[t][0]
        except:
            wID = words['<unk>'][0]
        corpus.append(wID)
    return(corpus)

def read_encode(file_name,vocab,words,corpus,threshold):
    
    wID = len(vocab)
    
    if threshold > -1:
        with open(file_name,'rt', encoding='utf8') as f:
            for line in f:
                line = line.replace('\n','')
                tokens = line.split(' ')
                for t in tokens:
                    try:
                        elem = words[t]
                    except:
                        elem = [wID,0]
                        vocab.append(t)
                        wID = wID + 1
                    elem[1] = elem[1] + 1
                    words[t] = elem

        temp = words
        words = {}
        vocab = []
        wID = 0
        words['<unk>'] = [wID,100]
        vocab.append('<unk>')
        for t in temp:
            if temp[t][1] >= threshold:
                vocab.append(t)
                wID = wID + 1
                words[t] = [wID,temp[t][1]]
            
                    
    with open(file_name,'rt', encoding='utf8') as f:
        for line in f:
            line = line.replace('\n','')
            tokens = line.split(' ')
            for t in tokens:
                try:
                    wID = words[t][0]
                except:
                    wID = words['<unk>'][0]
                corpus.append(wID)
                
    return [vocab,words,corpus]

In [3]:
# CUDA
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 

In [4]:
# FeedForward Model
class FFNN(nn.Module):
    def __init__(self, vocab, words, d_model, d_hidden, dropout):
        super().__init__() 
    
        # Class parameters
        self.vocab = vocab
        self.words = words
        self.vocab_size = len(self.vocab)
        self.d_model = d_model
        self.d_hidden = d_hidden
        self.dropout = dropout

        # Embedding Layer
        self.embeds = nn.Embedding(self.vocab_size, self.d_model)

        # Linear Layer
        self.fc1 = nn.Linear(d_model, d_hidden)

        # Nonlinear Layer
        self.activation = nn.ReLU()

    # Forward
    def forward(self, src):
        embeds = self.dropout(self.embeds(src))
        x = self.activation(self.fc1(embeds))
        return x
                
    def init_weights(self):
        pass

In [5]:
# LSTM Model
class LSTM(nn.Module):
    def __init__(self,vocab,words,d_model,d_hidden,n_layers,dropout_rate):
        super().__init__()
        
        # Class Parameters
        self.vocab = vocab
        self.words = words
        self.vocab_size = len(self.vocab)
        self.n_layers = n_layers
        self.d_hidden = d_hidden
        self.d_model = d_model

        # Embedding Layer
        self.embeds = nn.Embedding(self.vocab_size,self.d_model)
        
    # Forward
    def forward(self,src,h):
        embeds = self.dropout(self.embeds(src))       
        return [preds,h]
    
    def init_weights(self):
        pass        
    
    def detach_hidden(self, hidden):
        return [hidden, cell]

In [6]:
# Train Model Function
def train_model(model, data, targets, epochs):
    for _ in range(0, epochs):
        model.train()
        optim.zero_grad()

        output = model(data)

        loss = F.binary_cross_entropy(output, targets)
        loss.backward()

        optim.step()


In [7]:
class Params:
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)

In [8]:
model_map = {0: 'FFNN', 1: 'LSTM', 2: 'FFNN_CLASSIFY', 3: 'LSTM_CLASSIFY'}
train_map = {0: 'data/real.train.tok', 1: 'data/fake.train.tok', 2: 'data/mix.train.tok'}
valid_map = {0: 'data/real.valid.tok', 1: 'data/fake.valid.tok', 2: 'data/mix.valid.tok'}
test_map = {0: 'data/real.test.tok', 1: 'data/fake.test.tok', 2: 'data/mix.test.tok', 3: 'data/blind.test.tok'}

model_type = model_map[0]
train_type = [train_map[0], train_map[1]]
valid_type = valid_map[0]
test_type = test_map[0]

args = {
    "d_model": 1,
    "d_hidden": 2,
    "n_layers": 3,
    "batch_size": 20,
    "seq_len": 30,
    "printevery": 5000,
    "window": 3,
    "epochs": 20,
    "lr": 0.0001,
    "dropout": 0.35,
    "clip": 2.0,
    "model": model_type,
    "savename": model_type.lower(),
    "loadname": model_type.lower(),
    "trainname": train_type,
    "validname": valid_type,
    "testname": test_type
}

In [9]:
# Main Function
def main(args): 
    torch.manual_seed(0)
    
    params = Params(**args)
    # train_name = params.trainname
    test_name = params.testname
    model_type = params.model
    d_mod = params.d_model
    d_hid = params.d_hidden
    dropout = params.dropout
    epochs = params.epochs

    real, fake = params.trainname
    [vocab_real, words_real, train_real] = read_encode(real, [], {}, [], 3)
    [vocab_fake, words_fake, train_fake] = read_encode(fake, [], {}, [], 3)

    labels = torch.tensor()

    # [vocab,words,train] = read_encode(train_name,[],{},[],3)
    # print('vocab: %d train: %d' % (len(vocab),len(train)))
    # print(f'vocab: {vocab[10:20]}\n \n train: {train[10:20]}')
    # [vocab,words,test] = read_encode(test_name,vocab,words,[],-1)
    # print('vocab: %d test: %d' % (len(vocab),len(test)))
    # vocab_size = len(vocab)
    
    if model_type == 'FFNN':
        # ffnn_model = FFNN(vocab, words, d_mod, d_hid, dropout)
        # train_model(ffnn_model, data, targets, epochs)
        pass
        # print(ffnn_model)
#          {add code to instantiate the model, train for K epochs and save model to disk}
        
    if model_type == 'LSTM':
        pass
#          {add code to instantiate the model, train for K epochs and save model to disk}

    if model_type == 'FFNN_CLASSIFY':
        pass
#          {add code to instantiate the model, recall model parameters and perform/learn classification}

    if model_type == 'LSTM_CLASSIFY':
        pass
#          {add code to instantiate the model, recal l model parameters and perform/learn classification}


In [10]:
main(args)

vocab: 39352 train: 2881045
vocab: ['(', '12', 'may', '1901', '–', '19', 'july', '1983', ')', 'was']
 words: {'<unk>': [0, 100], '': [1, 79180], '<': [2, 15938], 'start_bio': [3, 7962], '>': [4, 15943], '=': [5, 15859], 'cecil': [6, 47], 'reginald': [7, 32], ',': [8, 148468], 'frs': [9, 240], '(': [10, 40596], '12': [11, 773], 'may': [12, 1892], '1901': [13, 504], '–': [14, 7352], '19': [15, 703], 'july': [16, 1578], '1983': [17, 330], ')': [18, 40569], 'was': [19, 45376], 'a': [20, 45981], 'british': [21, 1672], 'physicist': [22, 802], 'and': [23, 71979], 'engineer': [24, 387], '.': [25, 119681], 'he': [26, 49759], 'born': [27, 5057], 'in': [28, 87934], 'leeds': [29, 77], 'graduated': [30, 1225], 'from': [31, 14154], 'gonville': [32, 17], 'caius': [33, 32], 'college': [34, 3560], 'cambridge': [35, 981], '1923': [36, 582], 'began': [37, 1222], 'research': [38, 3750], 'at': [39, 23269], 'the': [40, 129195], 'metropolitan-vickers': [41, 4], 'company': [42, 755], 'trafford': [43, 3], 'par